Multiclass Classification 'Type'

In [2]:
!pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd
url = 'https://github.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/raw/refs/heads/main/Dynamically%20Generated%20Hate%20Dataset%20v0.2.3.csv'
df = pd.read_csv(url)
df.head(5)

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
4,5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,1,NaN


In [4]:
df1 = df[['text','type','split']]
df1.head(10)

,text,type,split
0,dalits are lowlives,notgiven,train
1,gay people are a burden to our society,notgiven,train
2,Arabs are not welcome,notgiven,train
3,I'm not saying we should actually eliminate he...,notgiven,train
4,bananas are for black people,notgiven,test
5,asian people are the shit,none,test
6,women can not reproduce kids but men can in al...,notgiven,test
7,i hate wearing black in the summer!,none,test
8,Who cares what Chinese people think?,notgiven,test
9,i try to punch a letter box when i see one,notgiven,test


In [5]:
df1['type'] = pd.Categorical(df1['type']).codes
df1['type'] = df1['type'] - df1['type'].min()

C:\Users\Intel6\AppData\Local\Temp\ipykernel_9924\3810510983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['type'] = pd.Categorical(df1['type']).codes
C:\Users\Intel6\AppData\Local\Temp\ipykernel_9924\3810510983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['type'] = df1['type'] - df1['type'].min()


In [6]:
print(df1['type'].unique())

[4 3 2 0 1 6 5]


In [7]:
df1[df1['type'] == 0]

,text,type,split
1263,travellers have a certain aroma,0,train
1274,yeah life is cunty but fucking hell get over i...,0,train
1289,the black women on my street have been heavily...,0,train
1365,"its amazing how much this area has got worse, ...",0,train
1370,"gay pride no longer comes through here, it sto...",0,train
...,...,...,...
34605,tell me what you have done EVER in your life t...,0,dev
34618,It seems wearing a uniform is not enough to st...,0,dev
34627,It amazes me how woman think they are the king...,0,dev
34633,Yeah. Probably why Trump didn't add a 10% fede...,0,dev


In [8]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Example data
texts = df1['text']
labels = df1['type'] # Categorical labels
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128

dataset = TextDataset(texts, labels, tokenizer, max_len)


In [9]:
train_texts = df1[df1['split'] == 'train']['text'].tolist()
train_labels = df1[df1['split'] == 'train']['type'].tolist()
val_texts = df1[df1['split'] == 'dev']['text'].tolist()
val_labels = df1[df1['split'] == 'dev']['type'].tolist()
test_texts = df1[df1['split'] == 'test']['text'].tolist()
test_labels = df1[df1['split'] == 'test']['type'].tolist()

# Assuming TextDataset is your custom dataset class
train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = TextDataset(val_texts, val_labels, tokenizer, max_len)
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_len)

In [10]:
num_labels = len(pd.Categorical(df1['type']).categories)
print(num_labels)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

7


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

  0%|          | 10/12348 [00:16<4:37:17,  1.35s/it]

{'loss': 2.0588, 'grad_norm': 11.828420639038086, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 20/12348 [00:31<5:36:50,  1.64s/it]

{'loss': 2.0831, 'grad_norm': 8.443136215209961, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  0%|          | 30/12348 [00:44<4:28:59,  1.31s/it]

...

100%|█████████▉| 12340/12348 [4:37:48<00:10,  1.35s/it]

{'loss': 0.3075, 'grad_norm': 6.308488845825195, 'learning_rate': 3.3760972316002705e-08, 'epoch': 3.0}


100%|██████████| 12348/12348 [4:38:00<00:00,  1.35s/it]

{'train_runtime': 16680.1167, 'train_samples_per_second': 5.922, 'train_steps_per_second': 0.74, 'train_loss': 0.6616929997981155, 'epoch': 3.0}


TrainOutput(global_step=12348, training_loss=0.6616929997981155, metrics={'train_runtime': 16680.1167, 'train_samples_per_second': 5.922, 'train_steps_per_second': 0.74, 'total_flos': 6497292959769600.0, 'train_loss': 0.6616929997981155, 'epoch': 3.0})

In [30]:
print(df1['target'].unique())

[310 308 130 357 259  46 324 378 373 120 315 227 218 170  28  19 250 168
 193 178   7 206 189   2 201 187 320 301  16 309 303  42  27   0 311 205
 147 118 264 287 409  44 300  50  45  69  26 323 226  75 249 175  94 317
 124 261   8  35 167 331   4 166 307 314 405 182 337 258 121  30 313 165
 156 325 327 328 102 101 145 296 140 136 134 255 388 348 265  65  52 159
 199   3 132  73 316 109 190 407 339 312  38 160 349 153 347 247 230 383
 372 254 392 285 232  78 216 267 333 141 246 277 391  58  63  62  72 257
 238 217 215  51  92 256 148  49 176 294 403 231 131  59  14 293  74 172
  37 229  98 214 185 204 297  31 384 400  81  82  91 371 343 406 399 390
 106 370 183 224 149 220 129 268 356 335 228 123  87 154 352  29 338 251
  66  85 144  86 212 395 353 169   5 234 289 126 380 171 326 162 355  68
 100  48 194  80 151  61  11 252 202 334 103  22 336 385 125 273 128 389
  90  60 318 393 329 196 288 158 117 236 398 110 319 115 394  57 188 275
 404 163 295 290 143 402 241 260  24 263 346 198 27

Replaced -1 with 409

In [26]:
df1['target'] = df1['target'].replace(-1, 409)

C:\Users\Intel6\AppData\Local\Temp\ipykernel_9984\1551034730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['target'] = df1['target'].replace(-1, 409)


In [33]:
df1[df['target'] == 409]

,text,target,split


In [34]:
invalid_rows = df1[df1['target'] == 409]
print(invalid_rows)

                                                    text  target  split
2220   I swear I will piss on this gross pikey until ...     409  train
2741   Really? There are a lot of gypsy travellers th...     409  train
2971   alot ogf people make fun of my personality in ...     409  train
4175   I have worse teeth than a camel humper, my mum...     409  train
9093    Please stop inviting this squaw at the workplace     409  train
9185   A girl I'm working with hails from Egypt and t...     409  train
11812                fucking hell I've just seen a pikey     409    dev
23651  The purpose of the Aryan Hall is not just to h...     409  train


In [12]:
model.save_pretrained('./results')

In [13]:
tokenizer.save_pretrained('./results')

('./results\\tokenizer_config.json',
 './results\\special_tokens_map.json',
 './results\\vocab.txt',
 './results\\added_tokens.json')

In [14]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('./results')
tokenizer = BertTokenizer.from_pretrained('./results')

In [12]:
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('./results')
tokenizer = BertTokenizer.from_pretrained('./results')

# Prepare your input text
text = "Insert text"
inputs = tokenizer(text, return_tensors='pt')

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probabilities = torch.nn.functional.softmax(logits, dim=-1)
category_mapping = dict(enumerate(pd.Categorical(df['target']).categories))

# Get the predicted class
predicted_class = torch.argmax(probabilities, dim=1).item()

print(f"Predicted class: {predicted_class} \n Targetted group: {category_mapping[predicted_class]}")

Predicted class: 3 
 Targetted group: arab, african
